In [1]:
import sys 
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from NN import network, activation_functions, loss_functions
from NN import ANN
X = np.array([[1,2],[3,4]])


## First Code running

In [2]:
# Random Init
teste = ANN.ANN("sigmoid")
teste.initialize_random_weights(2, [3], 2)
teste.show_weights()
teste.show_setup()
Y, aY = teste.foward_propagation(X)
print("Y\n",Y)
print("\na(Y)\n",aY)

Hidden Layer  1
[[0.49910255 0.86853766 0.20175092]
 [0.13500897 0.05661088 0.88562919]
 [0.85720335 0.45729096 0.08389379]] 

Output Layer 
[[0.41656271 0.64751417]
 [0.13480829 0.22349019]
 [0.46602173 0.02845841]
 [0.81667544 0.20098056]] 

 Input size:  2
 Number of hidden layers:  1
 Number of perceptrons at each layer: 
 HL 1: 3
 Number of classes: 2 

Y
 [[1.57728714 1.0324786 ]
 [1.77509366 1.0881012 ]]

a(Y)
 [[0.82881997 0.73739614]
 [0.85508998 0.748024  ]]


## Comparing with the new Implementation by using the same weights

In [3]:
from importlib import reload
reload(network)
I = np.array([[0.05, 0.10]])
W = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.35, label="H1")
netH, outH =  h1.feed_forward(I)
print(netH)
print(outH)

[[0.3775 0.3925]]
[[0.59326999 0.59688438]]


In [4]:
from importlib import reload
reload(network)
I = np.array([[0.59326999, 0.59688438]])
W = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.6, label="H1")
netO, outO =  o1.feed_forward(I)
print(netO)
print(outO)

[[1.10590597 1.2249214 ]]
[[0.75136507 0.77292847]]


In [5]:
from importlib import reload
reload(network)
model = network.NN(loss='smd')
X = np.array([[0.05, 0.10]])

Wh = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="H1")
model.add_layer(h)
model.add_layer(o)
model.show_weights()

Y, aY = model.feed_forward(X)
Y_, aY_ = ([1.10590597, 1.2249214 ], [0.75136507, 0.77292847])
print("Y\n",Y_)
print("\na(Y)\n",aY_)

assert((Y - Y_).sum() < np.finfo(np.float32).eps)
assert((aY - aY_).sum() < np.finfo(np.float32).eps)


Y
 [1.10590597, 1.2249214]

a(Y)
 [0.75136507, 0.77292847]


## Working on the back propagation

In [6]:
from importlib import reload
reload(network)
model.show_weights()
Y = np.array([0.01, 0.99])
Etotal = loss_functions.smd(aY, Y)
print(Etotal)
assert((Etotal - 0.2983711) < np.finfo(np.float32).eps)

0.2983711087600027


### Emulating the weights update for the layer O 

In [7]:
reload(loss_functions)
reload(activation_functions)
lr = 0.5
#Done - Partial
dEo_dw = loss_functions.smd_derivative_chain(outO, Y)
print(dEo_dw)

# Done
dOuto_Dneto = activation_functions.sigmoid_derivative_chain(outO)
print('dout',dOuto_Dneto)

# Done - Self.input
dNeto  = outH
print('dnet',dNeto) 
# Done
deltaO  = dEo_dw * dOuto_Dneto
print('delta', deltaO)

dWO = deltaO * outH
print('dw', dWO)
# Done
updateO = Wo - lr * dWO
print('update', updateO)

[[ 0.74136507 -0.21707153]]
dout [[0.1868156  0.17551005]]
dnet [[0.59326999 0.59688438]]
delta [[ 0.13849856 -0.03809824]]
dw [[ 0.08216704 -0.02274024]]
update [[0.35891648 0.51137012]
 [0.40891648 0.56137012]]


In [8]:
#
# Done - Partial
dEo_dOh = deltaO * Wo
dETotal_dOh = dEo_dOh.sum(axis=1)
print (dEo_dOh)
print('dETotal_dOh', dETotal_dOh)

# Done
dOuth_Dneth = activation_functions.sigmoid_derivative_chain(outH)
print('dOuth_Dneth', dOuth_Dneth)


# Done
deltaH = dETotal_dOh * dOuth_Dneth 
print('deltaH', deltaH)

# done
# self.input
dNeth_dw = X
dWh = deltaH * dNeth_dw
print('dWh', dWh)

# Done
updateH = Wh - lr * dWh
print('update', updateH)

[[ 0.05539942 -0.01904912]
 [ 0.06232435 -0.02095403]]
dETotal_dOh [0.03635031 0.04137032]
dOuth_Dneth [[0.24130071 0.24061342]]
deltaH [[0.00877135 0.00995425]]
dWh [[0.00043857 0.00099543]]
update [[0.14978072 0.24950229]
 [0.19978072 0.29950229]]


In [9]:
[i for i in range(8, -1, -1)]

[8, 7, 6, 5, 4, 3, 2, 1, 0]

## Layer by Layer testing

In [10]:
reload(loss_functions)
reload(activation_functions)
reload(network)
network.DEBUG = False

X = np.array([[0.05, 0.10]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="H1")
Y = np.array([0.01, 0.99], np.float64)

for i in range(10000):
    netH, outH = h.feed_forward(X)
#     print(netH, outH)
    netO, outO = o.feed_forward(outH)
#     print(netO, outO)
    
    Etotal = loss_functions.smd(outO, Y)
    if (i % 1000) == 0:
        print(i, 'error', Etotal)
    
    dEo_dw = loss_functions.smd_derivative_chain(outO, Y)
    network.dprint(dEo_dw)
    network.dprint ("")
    network.dprint ("==========================================")
    network.dprint ("Back Propagate Layer O")
    network.dprint ("==========================================")
    o.backpropagate(dETotal_dOut=dEo_dw)
    network.dprint ("==========================================")
    network.dprint ("")

    network.dprint ("==========================================")
    network.dprint ("Back Propagate Layer H")
    network.dprint ("==========================================")
    h.backpropagate(output_layer=o)
    network.dprint ("==========================================")

print (outO)

0 error 0.2983711087600027
1000 error 0.0011202377365627065
2000 error 0.00044785141631775586
3000 error 0.00025349028036380844
4000 error 0.00016571865888432704
5000 error 0.00011734553835230822
6000 error 8.7442721889668e-05
7000 error 6.75166200528868e-05
8000 error 5.351545683205839e-05
9000 error 4.32812038815559e-05
[[0.01600365 0.98407495]]


## Full Model Test

In [18]:
eps = 0.008

In [21]:
from utils import dataset_helper
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)
network.DEBUG = False

X = np.array([[0.05, 0.10]], np.float64)
Y = np.array([[0.01, 0.99]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="Output")

model = network.NN(loss='smd')
model.add_layer(h)
model.add_layer(o)
model.show_weights()

model.fit(X, Y, max_iter=50000, 
          lr=0.5, epsilon=eps, 
          print_interval=2000)
model.feed_forward(X)

It: 1 Batch: 1 Epoch 0 Error: 0.29837111 lr: 0.50000000 
Finished 
 It: 209 Batch: 1 Epoch 208 Train Loss: 0.00795372 lr: 0.50000000 


(array([[-2.11785483,  2.31889592]]), array([[0.1073735 , 0.91042995]]))

## Random initialization

### Softmax Sample and Test

In [24]:
from utils import dataset_helper
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)
network.DEBUG = False

X = np.array([[0.05, 0.10]], np.float64)
Y = np.array([[0.01, 0.99]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="Output")

model = network.NN(loss='cross_entropy')
model.add_layer(h)
model.add_layer(o)
model.show_weights()

model.fit(X, Y, max_iter=50000, 
          lr=0.5, epsilon=eps, 
          print_interval=2000)
model.feed_forward(X)

It: 1 Batch: 1 Epoch 0 Error: 1.65052853 lr: 0.50000000 
It: 2000 Batch: 1 Epoch 1999 Error: 0.13206266 lr: 0.50000000 
It: 4000 Batch: 1 Epoch 3999 Error: 0.12137317 lr: 0.50000000 
It: 6000 Batch: 1 Epoch 5999 Error: 0.11759200 lr: 0.50000000 
It: 8000 Batch: 1 Epoch 7999 Error: 0.11570863 lr: 0.50000000 
It: 10000 Batch: 1 Epoch 9999 Error: 0.11461245 lr: 0.50000000 
It: 12000 Batch: 1 Epoch 11999 Error: 0.11391401 lr: 0.50000000 
It: 14000 Batch: 1 Epoch 13999 Error: 0.11344174 lr: 0.50000000 
It: 16000 Batch: 1 Epoch 15999 Error: 0.11310871 lr: 0.50000000 
It: 18000 Batch: 1 Epoch 17999 Error: 0.11286639 lr: 0.50000000 
It: 20000 Batch: 1 Epoch 19999 Error: 0.11268576 lr: 0.50000000 
It: 22000 Batch: 1 Epoch 21999 Error: 0.11254852 lr: 0.50000000 
It: 24000 Batch: 1 Epoch 23999 Error: 0.11244259 lr: 0.50000000 
It: 26000 Batch: 1 Epoch 25999 Error: 0.11235979 lr: 0.50000000 
It: 28000 Batch: 1 Epoch 27999 Error: 0.11229435 lr: 0.50000000 
It: 30000 Batch: 1 Epoch 29999 Error: 0.11

(array([[-4.53180808,  4.53313307]]), array([[0.01064663, 0.98936732]]))

In [23]:
from utils import dataset_helper
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)
network.DEBUG = False

X = np.array([[0.05, 0.10]], np.float64)
Y = np.array([[0.01, 0.99]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="Output")

model = network.NN(loss='mse')
model.add_layer(h)
model.add_layer(o)
model.show_weights()

model.fit(X, Y, max_iter=50000, 
          lr=0.5, epsilon=eps, 
          print_interval=2000)
model.feed_forward(X)

It: 1 Batch: 1 Epoch 0 Error: 0.29837111 lr: 0.50000000 
Finished 
 It: 104 Batch: 1 Epoch 103 Train Loss: 0.00799325 lr: 0.50000000 


(array([[-2.11897627,  2.31855525]]), array([[0.10726606, 0.91040216]]))

## Testing the loss functions 

### Mean Squared Error

In [14]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])
lr = 0.01
eps  = 0.01
max_it = 100
for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.mse_derivative(I, aY, Y_)
    W = W - lr * dC

assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)
    


0 [ 2.7  8.1 16.2]
1 [ 2.48  7.44 14.88]
2 [ 2.282  6.846 13.692]
3 [ 2.1038  6.3114 12.6228]
4 [ 1.94342  5.83026 11.66052]
5 [ 1.799078  5.397234 10.794468]
6 [ 1.6691702  5.0075106 10.0150212]
7 [1.55225318 4.65675954 9.31351908]
8 [1.44702786 4.34108359 8.68216717]
9 [1.35232508 4.05697523 8.11395045]
10 [1.26709257 3.8012777  7.60255541]
11 [1.19038331 3.57114993 7.14229987]
12 [1.12134498 3.36403494 6.72806988]
13 [1.05921048 3.17763145 6.35526289]
14 [1.00328943 3.0098683  6.0197366 ]
15 [0.95296049 2.85888147 5.71776294]
16 [0.90766444 2.72299332 5.44598665]
17 [0.866898   2.60069399 5.20138798]
18 [0.8302082  2.49062459 4.98124919]
19 [0.79718738 2.39156213 4.78312427]
20 [0.76746864 2.30240592 4.60481184]
21 [0.74072178 2.22216533 4.44433066]
22 [0.7166496  2.1499488  4.29989759]
23 [0.69498464 2.08495392 4.16990783]
24 [0.67548617 2.02645852 4.05291705]
25 [0.65793756 1.97381267 3.94762534]
26 [0.6421438  1.9264314  3.85286281]
27 [0.62792942 1.88378826 3.76757653]
28 [0.615

### Cross Entropy 

In [15]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.cross_entropy_derivative(I, aY, Y_)
    W = W - lr * dC

assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)

0 [ 2.7  8.1 16.2]
1 [0.423 1.269 2.538]
2 [0.502695 1.508085 3.01617 ]


### Sum of Squared Errors 

In [16]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.smd_derivative(I, aY, Y_)
    W = W - lr * dC
assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)    

0 [ 2.7  8.1 16.2]
1 [0.423 1.269 2.538]
2 [0.502695 1.508085 3.01617 ]
